# حجز الفندق باستخدام وسيط عضو الأولوية

يُظهر هذا المفكرة **الوسيط المعتمد على الدالة** باستخدام إطار عمل Microsoft Agent. نبني على مثال سير العمل الشرطي عن طريق إضافة طبقة وسيط تمنح أعضاء الأولوية امتيازات خاصة.

## ما ستتعلمه:
1. **الوسيط المعتمد على الدالة**: اعتراض وتعديل نتائج الدالة  
2. **الوصول إلى السياق**: قراءة وتعديل `context.result` بعد التنفيذ  
3. **تنفيذ منطق الأعمال**: فوائد عضو الأولوية  
4. **تجاوز النتيجة**: تغيير نتائج الدالة بناءً على حالة المستخدم  
5. **نفس سير العمل، نتائج مختلفة**: تغييرات سلوك مدفوعة بالوسيط  

## بنية سير العمل مع الوسيط:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## الفرق الرئيسي عن سير العمل الشرطي:

**بدون وسيط** (14-conditional-workflow.ipynb):  
- باريس بلا غرف → التوجيه إلى alternative_agent  

**مع وسيط** (هذا المفكرة):  
- مستخدم عادي + باريس → بلا غرف → التوجيه إلى alternative_agent  
- عضو ذو أولوية + باريس → 🌟 الوسيط يتخطى! → متاح → التوجيه إلى booking_agent  

## المتطلبات المسبقة:
- تثبيت إطار عمل Microsoft Agent  
- فهم سير العمل الشرطي (انظر 14-conditional-workflow.ipynb)  
- رمز GitHub أو مفتاح API من OpenAI  
- الفهم الأساسي لأنماط الوسيط


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## الخطوة 1: تعريف نماذج Pydantic للمخرجات المهيكلة

تحدد هذه النماذج **المخطط** الذي ستقوم الوكلاء بإرجاعه. لقد أضفنا حقل `priority_override` لتعقب متى يقوم الوسيط بتعديل نتيجة التوافر.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## الخطوة 2: تعريف قاعدة بيانات أعضاء الأولوية

لهذا العرض التوضيحي، سنقوم بمحاكاة قاعدة بيانات عضوية الأولوية. في بيئة الإنتاج، سيكون هذا للاستعلام من قاعدة بيانات أو API حقيقية.

**أعضاء الأولوية:**
- `alice@example.com` - عضو مميز VIP
- `bob@example.com` - عضو بريميوم  
- `priority_user` - حساب اختبار


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## الخطوة 3: إنشاء أداة حجز الفندق

نفس سير العمل الشرطي، لكن الآن سيتم اعتراضه بواسطة الوسيط!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## الخطوة 4: 🌟 إنشاء وسيط التحقق من الأولوية (الميزة الأساسية!)

هذه هي **الوظيفة الأساسية** لهذا الدفتر. الوسيط:

1. **يعترض** استدعاء دالة hotel_booking
2. **ينفذ** الدالة بشكل طبيعي عن طريق استدعاء `next(context)`
3. **يفحص** النتيجة في `context.result`
4. **يستبدل** النتيجة إذا كان المستخدم ذو أولوية ولا توجد غرف متاحة
5. **يعيد** النتيجة المعدلة إلى الوكيل

**النمط الأساسي:**
```python
async def my_middleware(context, next):
    await next(context)  # تنفيذ الدالة
    # الآن يحتوي context.result على ناتج الدالة
    if some_condition:
        context.result = new_value  # تجاوز!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## الخطوة 5: تعريف دوال الشرط للتوجيه

نفس دوال الشرط المستخدمة في سير العمل الشرطي - تقوم بفحص الناتج المهيكل لتحديد التوجيه.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## الخطوة 6: إنشاء منفذ عرض مخصص

نفس المنفذ السابق - يعرض النتيجة النهائية لتدفق العمل.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## الخطوة 7: تحميل متغيرات البيئة

قم بتكوين عميل LLM (نماذج GitHub أو OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## الخطوة 8: إنشاء وكلاء الذكاء الاصطناعي مع الوسيط

**الفرق الرئيسي:** عند إنشاء availability_agent، نقوم بتمرير معلمة `middleware`!

هذه هي الطريقة التي نحقن بها priority_check_middleware في خط أنابيب استدعاء وظائف الوكيل.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## الخطوة 9: بناء سير العمل

نفس هيكل سير العمل كما من قبل - التوجيه الشرطي بناءً على التوفر.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## الخطوة 10: اختبار الحالة 1 - مستخدم عادي في باريس (بدون تجاوز)

يحاول مستخدم عادي حجز باريس → لا توجد غرف → يتم التوجيه إلى alternative_agent


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## الخطوة 11: الحالة الاختبارية 2 - 🌟 مستخدم ذو أولوية في باريس (مع التجاوز!)

يحاول عضو ذو أولوية حجز باريس → لا غرف في البداية → 🌟 الوسيط يتجاوز! → يوجه إلى booking_agent

**هذا هو العرض الرئيسي لقوة الوسيط!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## الخطوة 12: الحالة الاختبارية 3 - مستخدم ذو أولوية في ستوكهولم (متاح بالفعل)

المستخدم ذو الأولوية يجرب ستوكهولم → الغرف متاحة → لا حاجة للتجاوز → التوجيه إلى booking_agent

هذا يوضح أن الوسيط يتصرف فقط عند الحاجة!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## النقاط الرئيسية ومفاهيم الوسيط

### ✅ ما تعلمته:

#### **1. نمط الوسيط المعتمد على الدوال**

يقوم الوسيط باعتراض استدعاءات الدوال باستخدام دالة غير متزامنة بسيطة:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # قبل تنفيذ الدالة
    print("Intercepting...")
    
    # نفذ الدالة
    await next(context)
    
    # بعد تنفيذ الدالة - فحص النتيجة
    if context.result:
        # تعديل النتيجة إذا لزم الأمر
        context.result = modified_value
```

#### **2. الوصول إلى السياق وتجاوز النتيجة**

- `context.function` - الوصول إلى الدالة التي يتم استدعاؤها  
- `context.arguments` - قراءة وسيطات الدالة  
- `context.kwargs` - الوصول إلى المعلمات الإضافية  
- `await next(context)` - تنفيذ الدالة  
- `context.result` - قراءة/تعديل ناتج الدالة  

#### **3. تنفيذ منطق الأعمال**

يطبق وسيطنا مزايا الأعضاء ذوي الأولوية:  
- **المستخدمون العاديون**: لا تعديلات، سير عمل عادي  
- **المستخدمون ذوو الأولوية**: تجاوز "لا توجد توفر" → "متوفر"  
- **منطق مشروط**: يتجاوز فقط عند الحاجة  

#### **4. نفس سير العمل، نتائج مختلفة**

قوة الوسيط:  
- ✅ لا تغييرات على هيكل سير العمل  
- ✅ لا تغييرات على دالة الأداة  
- ✅ لا تغييرات على منطق التوجيه الشرطي  
- ✅ فقط وسيط → سلوك مختلف!

### 🚀 تطبيقات العالم الحقيقي:

1. **ميزات VIP/المميزة**  
   - تجاوز حدود المعدل للمستخدمين المميزين  
   - توفير وصول أولوية للموارد  
   - إلغاء قفل الميزات المميزة بشكل ديناميكي  

2. **اختبار A/B**  
   - توجيه المستخدمين إلى تطبيقات مختلفة  
   - اختبار ميزات جديدة مع مستخدمين محددين  
   - طرح الميزات تدريجياً  

3. **الأمان والامتثال**  
   - تدقيق استدعاءات الدوال  
   - حظر العمليات الحساسة  
   - تطبيق قواعد الأعمال  

4. **تحسين الأداء**  
   - تخزين النتائج مؤقتًا لمستخدمين محددين  
   - تخطي العمليات المكلفة عند الإمكان  
   - تخصيص الموارد بشكل ديناميكي  

5. **معالجة الأخطاء والمحاولة مرة أخرى**  
   - التقاط الأخطاء والتعامل معها بسلاسة  
   - تنفيذ منطق إعادة المحاولة  
   - العودة إلى تطبيقات بديلة  

6. **التسجيل والمراقبة**  
   - تتبع أوقات تنفيذ الدوال  
   - تسجيل المعطيات والنتائج  
   - مراقبة أنماط الاستخدام  

### 🔑 الفروقات الرئيسية عن المزخرفات:

| الميزة | المزخرف | الوسيط |
|---------|-----------|------------|
| **النطاق** | دالة واحدة | كل الدوال في الوكيل |
| **المرونة** | ثابت عند التعريف | ديناميكي عند وقت التشغيل |
| **السياق** | محدود | سياق الوكيل الكامل |
| **التكوين** | عدة مزخرفات | خط أنابيب الوسيط |
| **الإدراك بالوكيل** | لا | نعم (الوصول إلى حالة الوكيل) |

### 📚 متى تستخدم الوسيط:

✅ **استخدم الوسيط عندما:**  
- تحتاج إلى تعديل السلوك بناءً على حالة المستخدم/الجلسة  
- ترغب في تطبيق المنطق على عدة دوال  
- تحتاج الوصول إلى سياق مستوى الوكيل  
- تقوم بتنفيذ اهتمامات عابرة (التسجيل، المصادقة، إلخ)  

❌ **لا تستخدم الوسيط عندما:**  
- تحقق بسيط من المدخلات (استخدم Pydantic)  
- منطق خاص بالدالة (احتفظ به في الدالة)  
- تعديلات لمرة واحدة (قم بتغيير الدالة فقط)  

### 🎓 أنماط متقدمة:

```python
# وسيطات متعددة (ترتيب التنفيذ مهم!)
middleware=[
    logging_middleware,      # يسجل أولاً
    auth_middleware,         # ثم يتحقق من المصادقة
    cache_middleware,        # ثم يتحقق من التخزين المؤقت
    rate_limit_middleware,   # ثم يحدد حدود المعدل
    priority_check_middleware  # أخيرًا تحقق الأولوية
]

# تنفيذ الوسيطات بشكل مشروط
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # تعديل النتيجة
    else:
        # تخطي التنفيذ تمامًا
        context.result = cached_value
```

### 🔗 مفاهيم مرتبطة:

- **وسيط الوكيل**: يعترض استدعاءات agent.run()  
- **وسيط الدالة**: يعترض استدعاءات دوال الأداة (ما استخدمناه!)  
- **خط أنابيب الوسيط**: سلسلة من الوسائط تنفذ بالتسلسل  
- **انتشار السياق**: تمرير الحالة عبر سلسلة الوسيطات


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**إخلاء المسؤولية**:
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق به. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية محترفة. نحن غير مسؤولين عن أي سوء فهم أو تفسير ناتج عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
